In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from transformers import AutoTokenizer, AutoModel
import os
import matplotlib.pyplot as plt



In [ ]:


# 加载嵌入
embedding_file = '/mnt/data/lzp/OGT/output/protein_embeddings.pt'
X_embeddings = torch.load(embedding_file)

# 将嵌入数据转换为 NumPy 数组
if isinstance(X_embeddings, list):
    X_embeddings = [torch.tensor(embed) for embed in X_embeddings]
    X = torch.cat(X_embeddings, dim=0).numpy()
else:
    X = X_embeddings.numpy()


/tmp/ipykernel_1019743/1205574855.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_embeddings = torch.load(embedding_file)


In [14]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ==== 设置设备 ====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ==== 加载数据 ====
embedding_file = '/mnt/data/lzp/OGT/output/protein_embeddings.pt'
X_embeddings = torch.load(embedding_file)

if isinstance(X_embeddings, list):
    X_embeddings = [torch.tensor(embed) for embed in X_embeddings]
    X = torch.cat(X_embeddings, dim=0).numpy()
else:
    X = X_embeddings.numpy()
    
# 加载物理化学特征并忽略第一行作为列名
protein_features_file = '/mnt/data/lzp/OGT/result/ifeature_result/protein_features.csv'
protein_features = pd.read_csv(protein_features_file, header=1)  # 从第二行开始读取数据
# 删除第一列
protein_features = protein_features.drop(protein_features.columns[0], axis=1)
protein_features = protein_features.iloc[:X.shape[0], :]  # 保留前 93220 行
X_combined = np.concatenate([X, protein_features.values], axis=1)
print("X_combined shape:", X_combined.shape)
print("第1行第4373列的值是:", X_combined[0, 4372])


labels_file = '/mnt/data/lzp/OGT/output/protein_labels.csv'
y = pd.read_csv(labels_file)['ogt'].values

# ==== 划分数据 ====
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


Using device: cuda


/tmp/ipykernel_698839/3050903343.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_embeddings = torch.load(embedding_file)


X_combined shape: (93220, 4373)
第1行第4373列的值是: 99.23076923076924


In [15]:
print(X.shape)
print(y.shape)

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(93220, 1280)
(93220,)
(74576, 4373)
(18644, 4373)
(74576,)
(18644,)


In [3]:
print(X[:2])

[[ 0.13919732 -0.02285788  0.05318638 ... -0.22690704  0.10845955
  -0.05633758]
 [ 0.11506157 -0.02977298  0.05750177 ... -0.23991719  0.11758859
  -0.07531863]]


In [4]:
# 加载物理化学特征并忽略第一行作为列名
protein_features_file = '/mnt/data/lzp/OGT/result/ifeature_result/protein_features.csv'
protein_features = pd.read_csv(protein_features_file, header=1)  # 从第二行开始读取数据

print(protein_features.head(2))  # 查看前两行数据


   protein_1  0.0899280575539568  0.0071942446043165  0.0539568345323741  \
0  protein_2            0.103846            0.011538            0.046154   
1  protein_3            0.077586            0.014368            0.040230   

   0.1151079136690647  0.0287769784172661  0.079136690647482  \
0            0.084615            0.023077           0.088462   
1            0.077586            0.034483           0.103448   

   0.0179856115107913  0.0683453237410072  0.079136690647482.1  ...  \
0            0.015385            0.084615             0.065385  ...   
1            0.017241            0.068966             0.071839  ...   

   0.7194244604316548.7  25.179856115107917.3  51.43884892086332.1  \
0              1.923077             21.153846            42.307692   
1              0.862069             24.137931            48.563218   

   76.97841726618705.3  99.28057553956836.11  0.3597122302158274.12  \
0            75.384615                 100.0               0.384615   
1          

In [5]:
# 删除第一列
protein_features = protein_features.drop(protein_features.columns[0], axis=1)
print(protein_features.head(2))  # 查看前两行数据


   0.0899280575539568  0.0071942446043165  0.0539568345323741  \
0            0.103846            0.011538            0.046154   
1            0.077586            0.014368            0.040230   

   0.1151079136690647  0.0287769784172661  0.079136690647482  \
0            0.084615            0.023077           0.088462   
1            0.077586            0.034483           0.103448   

   0.0179856115107913  0.0683453237410072  0.079136690647482.1  \
0            0.015385            0.084615             0.065385   
1            0.017241            0.068966             0.071839   

   0.0863309352517985  ...  0.7194244604316548.7  25.179856115107917.3  \
0            0.073077  ...              1.923077             21.153846   
1            0.074713  ...              0.862069             24.137931   

   51.43884892086332.1  76.97841726618705.3  99.28057553956836.11  \
0            42.307692            75.384615                 100.0   
1            48.563218            73.275862        

In [6]:
print(protein_features.shape)


(93320, 3093)


In [7]:
protein_features = protein_features.iloc[:X.shape[0], :]  # 保留前 93220 行

In [8]:
X_combined = np.concatenate([X, protein_features.values], axis=1)

In [9]:
print("X_combined shape:", X_combined.shape)
print("第1行第4373列的值是:", X_combined[0, 4372])


X_combined shape: (93220, 4373)
第1行第4373列的值是: 99.23076923076924


In [13]:
# 加载标签
labels_file = '/mnt/data/lzp/OGT/output/protein_labels.csv'
y = pd.read_csv(labels_file)['ogt'].values

# 数据分割
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:

# 定义一个更深的深度神经网络 (DNN) 模型
class DNNModel(nn.Module):
    def __init__(self, input_dim):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)  # 添加 Batch Normalization
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 1)  # 输出为一个标量，回归任务
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # 增加 Dropout 比例
    
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))  # 加入 Batch Normalization
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))  # 加入 Batch Normalization
        x = self.dropout(x)
        x = self.relu(self.bn3(self.fc3(x)))  # 加入 Batch Normalization
        x = self.fc4(x)
        return x

# 将训练数据转为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # 回归任务，输出为一维
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 创建 DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # 调整批量大小
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 初始化模型
input_dim = X_train.shape[1]  # 特征维度
model = DNNModel(input_dim)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)  # 加入 L2 正则化

# 训练模型
epochs = 300  # 增加训练轮数
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 测试模型
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# 将预测结果转换为 numpy 数组
y_pred = y_pred_tensor.numpy()

# 计算评价指标
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 输出评价指标
print("\n深度学习回归模型评价指标：")
print(f"均方误差 (MSE): {mse:.2f}")
print(f"决定系数 (R²): {r2:.2f}")

# 可视化实际值与预测值的对比
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6, color='blue', label='预测值')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label='理想预测')
plt.xlabel("实际值")
plt.ylabel("预测值")
plt.title("深度学习回归模型：实际值 vs 预测值")
plt.legend()
plt.grid(True)
plt.show()